In [1]:
!pip install pandas
import pandas as pd
df = pd.read_csv("./signed.csv", parse_dates=True)
!pip install protobuf==3.19.6
!pip install tensorflow-gpu==1.4.0

You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [22]:
import tensorflow as tf
tf.__version__
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
!pip install gensim
from gensim.models import Word2Vec
import time

sentences = []
for i in range(df.shape[0]):
    sentences.append(df['Text'].iloc[i].split(' '))
start = time.time()
model = Word2Vec(sentences=sentences, size=50, window=5, min_count=1, workers=20)
model.save("word2vec.model")
end = time.time()
print(end - start)

You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


109.2319860458374


In [4]:
print(model.wv.most_similar('газ'))

[('нефт', 0.824609637260437), ('электроэнерг', 0.8146584033966064), ('энергоресурс', 0.7953792810440063), ('энергоносител', 0.7818318009376526), ('топлив', 0.7719652056694031), ('спг', 0.7435731887817383), ('газов', 0.7391821146011353), ('нефтепродукт', 0.7325606346130371), ('газа»', 0.7177338600158691), ('пхг', 0.7114875316619873)]


In [5]:
vectors = []
for text in df['Text']:
    vectors.append([model.wv[w] for w in text.split(' ')])

In [6]:
import numpy as np
import tensorflow as tf

X = vectors
y = np.zeros((len(vectors), 3))
classes = [-1, 0, 1]
for i in range(len(vectors)):
    y[i, classes.index(df['ChangeSign'].iloc[i])] = 1

In [11]:
count = int(len(vectors) * 0.8)
trainX, testX = X[:count], X[count:]
trainY, testY = y[:count], y[count:]

In [29]:
# Creates a session with log_device_placement set to True.
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device("GPU: 0"):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
        c = tf.matmul(a, b)
        sess.run(tf.global_variables_initializer())
        print(sess.run(c))
        print(a.device)
        print(type(a.shape[0]))

[[22. 28.]
 [49. 64.]]
/device:GPU:0
<class 'tensorflow.python.framework.tensor_shape.Dimension'>


In [70]:
lstm = tf.keras.layers.LSTM(3, input_shape=(1, 50))
model = tf.keras.models.Sequential([lstm])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
hidden_state = np.random.rand(1, 3)
cell_state = np.random.rand(1, 3)
for epoch in range(1):
    for i in range(len(trainX)):
        print(i)
        xs = X[i]
        y = trainY[i]
        y = np.reshape(y, [1, -1])
        cy = tf.constant(y)
        start = time.time()
        for x in xs: 
            x = np.reshape(np.array(x), [1, 1, 50])
            model.layers[0].states[0] = hidden_state
            model.layers[0].states[1] = cell_state
            cx = tf.constant(x)
            history = model.fit(chx, chy, epochs=1, verbose=0)
        end = time.time()
        print(end - start)
        print(history.history)

0
<class 'tensorflow.python.framework.tensor_shape.TensorShape'>


TypeError: unhashable type: 'Dimension'